In [27]:
%pip install rubik-cube numpy matplotlib pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [28]:
from rubik.cube import Cube

def colored_cube(cube: Cube):
  cube_str = ""
  for char in cube.__str__():
      if(char == "O"):
        cube_str += "\033[38;5;214mO\033[0m"
      elif(char == "Y"):
        cube_str += "\033[38;5;11mY\033[0m"
      elif(char == "W"):
        cube_str += "\033[38;5;15mW\033[0m"
      elif(char == "G"):
        cube_str += "\033[38;5;10mG\033[0m"
      elif(char == "B"):
        cube_str += "\033[38;5;12mB\033[0m"
      elif(char == "R"):
        cube_str += "\033[38;5;9mR\033[0m"
      else:
        cube_str += char
  
  return cube_str

c = Cube("OOOOOOOOOYYYWWWGGGBBBYYYWWWGGGBBBYYYWWWGGGBBBRRRRRRRRR")

print(colored_cube(c))


    OOO
    OOO
    OOO
YYY WWW GGG BBB
YYY WWW GGG BBB
YYY WWW GGG BBB
    RRR
    RRR
    RRR


In [29]:
import random

SOLVED_CUBE_STR = "OOOOOOOOOYYYWWWGGGBBBYYYWWWGGGBBBYYYWWWGGGBBBRRRRRRRRR"
MOVES = ["L", "R", "U", "D", "F", "B", "M", "E", "S"]


def random_cube(num_moves: int = 1, seq: bool = False):
    """
    :return: A new scrambled Cube
    """
    scramble_moves = " ".join(random.choices(MOVES, k=num_moves))
    c = Cube(SOLVED_CUBE_STR)
    c.sequence(scramble_moves)
    if seq:
        return c, scramble_moves
    return c

print(colored_cube(random_cube(1)))

    OOO
    OOO
    YYY
YYR WWW OGG BBB
YYR WWW OGG BBB
YYR WWW OGG BBB
    GGG
    RRR
    RRR


In [30]:
# def cube_hash(cube: Cube):
#   res = 0
#   res_list=[]
#   for piece in cube._color_list():
#     # O: 1, Y: 2, W: 3, G: 4, B: 5, R: 6
#     if(piece == "O"):
#       res_list.append(1)
#     elif(piece == "Y"):
#       res_list.append(2)
#     elif(piece == "W"):
#       res_list.append(3)
#     elif(piece == "G"):
#       res_list.append(4)
#     elif(piece == "B"):
#       res_list.append(5)
#     elif(piece == "R"):
#       res_list.append(6)
  
#   for i in range(6):
#       res *= 9
#       res += res_list[i]
  
#   return res
import hashlib

def cube_hash(cube: Cube):
  return hashlib.sha1(cube.__str__().encode("UTF-8")).hexdigest()[:10]

c = random_cube(100)
print(colored_cube(c))
print("Cube Hash:", cube_hash(c))
# c = Cube("OOOOOOOOOYYYWWWGGGBBBYYYWWWGGGBBBYYYWWWGGGBBBRRRRRRRRR")

# print("Cube Hash:", cube_hash(c)[:10])

    WYB
    BBG
    YBW
OOO BYR GWO GOG
BOB GGY RRW RYR
RGO WWY RWY WGB
    YYB
    OWO
    GRR
Cube Hash: 9e3d5de5d2


In [31]:
cube = Cube("OOOOOOOOOYYYWWWGGGBBBYYYWWWGGGBBBYYYWWWGGGBBBRRRRRRRRR")
cube.S()
print(cube)
print(cube.is_solved())
cube.Si()
print(cube)
print(cube.is_solved())

    OOO
    YYY
    OOO
YRY WWW GOG BBB
YRY WWW GOG BBB
YRY WWW GOG BBB
    RRR
    GGG
    RRR
False
    OOO
    OOO
    OOO
YYY WWW GGG BBB
YYY WWW GGG BBB
YYY WWW GGG BBB
    RRR
    RRR
    RRR
True


In [32]:
import numpy as np

MOVES_I = ["Li", "Ri", "Ui", "Di", "Fi", "Bi", "Mi", "Ei", "Si"]

class Agent:
    def __init__(self, alpha=0.9, gamma=0.95, q_init=0.6):
        # alpha: The learning rate needs to be larger than 0 and smaller than 1
        # gamma: The reward discount. Needs to be larger than 0  and should be smaller than 1. Values close to 1 should work best.
        # q_init: The initial q values for each action and state.
        
        self.q = {}  # type: Dict[int, np.ndarray]
        self.action_history = []  # type: List[(int, int)]
        self.learning_rate = alpha
        self.value_discount = gamma
        self.q_init_val = q_init
        super().__init__()

    def get_q(self, hash_value: int) -> np.ndarray:
        # Returns the q values for the state with hash value of cube
        # hash_value: The hash value of the board state for which the q values should be returned
        # return: List of q values for the input state hash.

        #
        # Build Q table in a lazy manner, only adding a state when it is actually used for the first time
        #
        if hash_value in self.q:
            qvals = self.q[hash_value]
        else:
            qvals = np.full(9, self.q_init_val)
            self.q[hash_value] = qvals

        return qvals

    def get_action(self, cube: Cube) -> int:
        # Return the next action given the cube `board` cube on the current Q values
        # cube: The current cube state
        # return: The next action basesd on the current Q values for the input state
        
        hash_value = cube_hash(cube)  # type: int
        qvals = self.get_q(hash_value)  # type: np.ndarray
        m = np.argmax(qvals)  # type: int
        return m

    def action(self, cube: Cube):
        # Makes a action to solve the cube
        # cube: The cube to make a action on
        # return: Bool whether the cube is solved

        m = self.get_action(cube)
        self.action_history.append((cube_hash(cube), m))
        # cube.sequence(MOVES[m])
        cube.sequence(MOVES_I[m])

        solved = cube.is_solved()

        return solved, MOVES_I[m]

    def update_q(self, moves: int = 0, result: float = None):
        # Gets called after the game has finished. Will update the current Q function based on the actions.

        if(result == None):
            result =  len(self.action_history) / moves
    
        # if(len(self.action_history) == moves):
        #     result = 1
        # else:
        #     result = 0.5

        self.action_history.reverse()
        next_max = -1.0  # type: float

        for a in self.action_history:
            qvals = self.get_q(a[0])
            if next_max < 0:  # First time through the loop
                qvals[a[1]] = result
            else:
                qvals[a[1]] = qvals[a[1]] * (
                            1.0 - self.learning_rate) + self.learning_rate * self.value_discount * next_max

            next_max = max(qvals)

    def new_agent(self):
        self.action_history = []

In [33]:
def try_to_solve(cube: Cube, agent: Agent, num_moves: int, cancle_after: int) -> bool:
    agent.new_agent()
    
    for _ in range(cancle_after):
      solved, move = agent.action(cube)
      if solved:
        agent.update_q(num_moves)
        return True, move

    agent.update_q(num_moves, -1)
    return False, move

In [54]:
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt

now = datetime.now().strftime("%d%m%Y_%H%M%S")
file_name = f'data/cube_{now}.csv'

def train(agent: Agent, num_moves: int = 1, cancle_after: int = 1, num_games: int = 10, log: bool = False):

  if log:
    row=[]
    rows=[]

  solved_count=0
  for _ in range(num_games):
    cube, seq = random_cube(num_moves, True)
    # cube = Cube("OOOOOOOOOYYYWWWGGGBBBYYYWWWGGGBBBYYYWWWGGGBBBRRRRRRRRR")
    # cube.S()
    if log:
      row.append(cube_hash(cube))
      row.append(seq)

    solved, move = try_to_solve(cube, agent, num_moves, cancle_after)

    if log:
      row.append(move)
      row.append(solved)
      row.append(agent.q)
      rows.append(row)
      row=[]

    if(solved):
      solved_count+=1

  if log:
    df_cubes = pd.DataFrame(rows, columns=['Hash', 'Action', 'Move', 'Solved', "Q"])
    df_cubes.to_csv(file_name, index=False, sep="|")
  print(solved_count)

log = False
agent = Agent()
train(agent, 4, 4, 15000, log)
if log:
  a = pd.read_csv(file_name, sep="|")

  plt.ylabel('Gelöst')
  plt.xlabel('Spiele')
  plt.plot([ind for ind, x in enumerate(a["Solved"].array)], a["Solved"].array.astype(int)   , 'g-')
  plt.legend(shadow=True, fancybox=True, framealpha =0.7)

# a


217


In [35]:
from datetime import datetime
# 
now = datetime.now().strftime("%d%m%Y_%H%M%S")
print("date and time =", now)

date and time = 11052022_232644


In [36]:
import pandas as pd

df = pd.read_csv('data/test.csv')
print(df["a"].array)

FileNotFoundError: [Errno 2] No such file or directory: 'data/test.csv'

In [ ]:
cities = pd.DataFrame([['Sacramento', 'California'], ['Miami', 'Florida']], columns=['City', 'State'])

cubes = [random_cube(4) for _ in range(5) ]
# [print(cube) for cube in cubes]
cube_hashes = [cube_hash(cube) for cube in cubes]
colored_cubes = [colored_cube(cube) for cube in cubes]
df = [[colored_cubes[i], cube_hashes[i]] for i in range(len(cubes))]
# print(df)

df_cubes = pd.DataFrame(df, columns=['Cube', 'Hash'])

df_cubes.to_csv('data/cube.csv', index=False, sep="|")
a = pd.read_csv('data/cube.csv', sep="|")

print(a)
from IPython.display import display, HTML
# display(HTML(a.to_clipboard()))
# print(a.to_json())
# print(a)
# for column in a:
#     print(a[column])
# for _ in range(len(cubes)):
#   for name, values in a.iteritems():
#       print('{name}: {value}'.format(name=name, value=values[0]))
# for (columnName, columnData) in a.iteritems():
#     print('Column Name : ', columnName)
#     print('Column Contents : ', columnData.values[0])
# for x in a["Cube"]:
#     print(x)
!pip3 install ansi2html
pd.set_option('display.max_colwidth', None)
from IPython.display import display, HTML
from ansi2html import Ansi2HTMLConverter
conv = Ansi2HTMLConverter()

def pretty_print(df):
    for i, x in enumerate(df["Cube"]):
        df["Cube"][i] = conv.convert("".join(x))
        # print(display(HTML(conv.convert(x))))

    html_df = df.to_html()
    newline_df = html_df.replace("\\n","<br>")
    # colored_df = conv.convert(newline_df)


    return  display(HTML(newline_df))

display(HTML('<h1>Hello, world!</h1>'))
pretty_print(a)
# display(HTML(a.to_html()))

# 


# conv = Ansi2HTMLConverter()
# ansi = "".join(colored_cube(random_cube(1)))
# html = conv.convert(ansi)
# display( HTML( html ) )

                                                Cube        Hash
0      WO...  65f0760865
1      YGG...  c9a66a159b
2      OO...  1117572ea8
3      ORO...  40643657bf
4      YBB...  89f3b87ebc
Defaulting to user installation because normal site-packages is not writeable


,Cube,Hash
0,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 Transitional//EN"" ""http://www.w3.org/TR/html4/loose.dtd""><html><head><meta http-equiv=""Content-Type"" content=""text/html; charset=utf-8""><title></title><style type=""text/css"">.ansi2html-content { display: inline; white-space: pre-wrap; word-wrap: break-word; }.body_foreground { color: #AAAAAA; }.body_background { background-color: #000000; }.body_foreground > .bold,.bold > .body_foreground, body.body_foreground > pre > .bold { color: #FFFFFF; font-weight: normal; }.inv_foreground { color: #000000; }.inv_background { background-color: #AAAAAA; }.ansi38-9 { color: #ff0000; }.ansi38-10 { color: #00ff00; }.ansi38-11 { color: #ffff00; }.ansi38-12 { color: #5c5cff; }.ansi38-15 { color: #ffffff; }.ansi38-214 { color: #ffaf00; }</style></head><body class=""body_foreground body_background"" style=""font-size: normal;"" ><pre class=""ansi2html-content""> <span class=""ansi38-15"">W</span><span class=""ansi38-214"">O</span><span class=""ansi38-214"">O</span> <span class=""ansi38-12"">B</span><span class=""ansi38-214"">O</span><span class=""ansi38-214"">O</span> <span class=""ansi38-12"">B</span><span class=""ansi38-11"">Y</span><span class=""ansi38-11"">Y</span><span class=""ansi38-214"">O</span><span class=""ansi38-11"">Y</span><span class=""ansi38-11"">Y</span> <span class=""ansi38-214"">O</span><span class=""ansi38-15"">W</span><span class=""ansi38-15"">W</span> <span class=""ansi38-214"">O</span><span class=""ansi38-10"">G</span><span class=""ansi38-10"">G</span> <span class=""ansi38-12"">B</span><span class=""ansi38-12"">B</span><span class=""ansi38-10"">G</span><span class=""ansi38-10"">G</span><span class=""ansi38-11"">Y</span><span class=""ansi38-11"">Y</span> <span class=""ansi38-214"">O</span><span class=""ansi38-15"">W</span><span class=""ansi38-15"">W</span> <span class=""ansi38-214"">O</span><span class=""ansi38-10"">G</span><span class=""ansi38-10"">G</span> <span class=""ansi38-12"">B</span><span class=""ansi38-12"">B</span><span class=""ansi38-9"">R</span><span class=""ansi38-10"">G</span><span class=""ansi38-9"">R</span><span class=""ansi38-9"">R</span> <span class=""ansi38-11"">Y</span><span class=""ansi38-12"">B</span><span class=""ansi38-12"">B</span> <span class=""ansi38-11"">Y</span><span class=""ansi38-11"">Y</span><span class=""ansi38-9"">R</span> <span class=""ansi38-15"">W</span><span class=""ansi38-15"">W</span><span class=""ansi38-9"">R</span> <span class=""ansi38-15"">W</span><span class=""ansi38-9"">R</span><span class=""ansi38-9"">R</span> <span class=""ansi38-15"">W</span><span class=""ansi38-9"">R</span><span class=""ansi38-9"">R</span> <span class=""ansi38-12"">B</span><span class=""ansi38-10"">G</span><span class=""ansi38-10"">G</span></pre></body></html>",65f0760865
1,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 Transitional//EN"" ""http://www.w3.org/TR/html4/loose.dtd""><html><head><meta http-equiv=""Content-Type"" content=""text/html; charset=utf-8""><title></title><style type=""text/css"">.ansi2html-content { display: inline; white-space: pre-wrap; word-wrap: break-word; }.body_foreground { color: #AAAAAA; }.body_background { background-color: #000000; }.body_foreground > .bold,.bold > .body_foreground, body.body_foreground > pre > .bold { color: #FFFFFF; font-weight: normal; }.inv_foreground { color: #000000; }.inv_background { background-color: #AAAAAA; }.ansi38-9 { color: #ff0000; }.ansi38-10 { color: #00ff00; }.ansi38-11 { color: #ffff00; }.ansi38-12 { color: #5c5cff; }.ansi38-15 { color: #ffffff; }.ansi38-214 { color: #ffaf00; }</style></head><body class=""body_foreground body_background"" style=""font-size: normal;"" ><pre class=""ansi2html-content""> <span class=""ansi38-11"">Y</span><span class=""ansi38-10"">G</span><span class=""ansi38-10"">G</span> <span class=""ansi38-9"">R</span><span class=""ansi38-11"">Y</span><span class=""ansi38-214"">O</span> <span class=""ansi38-11"">Y</span><span class=""ansi38-11"">Y</span><spa